# Add Taxonomy

This kinda sorta works, but turns out that the lookups aren't always exact. It kinda hammers the
PBDB, too, which probably isn't very neighborly.

Libraries.

In [70]:
import os
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict

This is a helper function that will get the taxonomy for a given species from the PBDB. It will return a dataframe object that corresponds to the returned stuff from PBDB.

In [84]:
def get_PBDB_taxonomy_df(id_name):
    # Format string for use in a URL. No spaces.
    id_name = id_name.replace(' ','%20')
    url = f'http://paleobiodb.org/data1.2/taxa/list.csv?datainfo&rowcount&base_name={id_name}&show=full'
    r = requests.get(url, allow_redirects=True)
    tmp_fname = 'pbdb_tax_tmp.csv'
    tmp_tax_file = open(tmp_fname, 'wb')
    tmp_tax_file.write(r.content)
    tmp_tax_file.close()
    pbdb_taxonomy_df = pd.read_csv(tmp_fname, low_memory=False, encoding='latin1', skiprows=15)
    if len(pbdb_taxonomy_df.columns) > 1:
        return pbdb_taxonomy_df
    else:
        return pd.DataFrame({'A' : []})

kaili_species_list_df = pd.read_csv('../Kaili species list.csv', encoding='latin1', dtype='string')

for i in kaili_species_list_df.index:
    something_missing = (
        pd.isnull(kaili_species_list_df.loc[i,'class']) or
        pd.isnull(kaili_species_list_df.loc[i,'order']) or 
        pd.isnull(kaili_species_list_df.loc[i,'family']) or 
        pd.isnull(kaili_species_list_df.loc[i,'genus'])
        )
    if something_missing:
        try:
            tax_df = get_PBDB_taxonomy_df(kaili_species_list_df.loc[i, 'identified_name'])
            if not tax_df.empty:
                kaili_species_list_df.loc[i,'class'] = tax_df.loc[0,'class']
                kaili_species_list_df.loc[i,'order'] = tax_df.loc[0,'order']
                kaili_species_list_df.loc[i,'family'] = tax_df.loc[0,'family']
                kaili_species_list_df.loc[i,'genus'] = tax_df.loc[0,'genus']
                time.sleep(0.5)
        except:
            pass
    print(i)
    print(kaili_species_list_df.loc[i, 'identified_name'])
            
print('done')


0
Marpolia spissa 
1
Marpolia aequalis
2
Morania confluens
3
Morania sp. 
4
Acinocricus stichus
5
Bosworthia gyges
6
Bosworthia simulans
7
Dalyia racemata
8
Doushantuophyton cometa
9
Doushantuophyton lineare
10
Enteromophites sinianasis
11
Eolaminia grandis
12
Eolaminia perelegans
13
Eosargassum sawata
14
Eoulothrix fibrillata
15
Eoulothrix flabellophyton
16
Flabelliphyton lantianensis
17
Fractibeltia fibrillata
18
Fractibeltia formosus
19
Fractibeltia typica (= F. typical in Zhao et al., 2002, fig. 6-16i)
20
Fractibeltia vein
21
Guizhouella ramulosa
22
Kailiphyton regularum
23
Kailiphyton simulans
24
Konglingiphyton erecta
25
Leafiophyton dichuarae
26
Palaeocodium yichangicum
27
Palaeodictyota dichotoma
28
Palaeospinella typica (P. typical in Zhao et al., 2002, table 6-1)
29
Palmalga glumacea
30
Paraamphiros siniansis 
31
Rhizophyton zhaouanlongii 
32
Sinocylindra yunnanensis Sinocylindra sp. 
33
Thamnophton formosus (= T. fomostus; Zhao et al., 2002, fig. 6-15)
34
Udotealga erecta
35

In [85]:
# dump df to CSV file.
kaili_species_list_df.to_csv('../Kaili species list taxdata.csv', index=False, sep=',', encoding='utf-8')
